# Дообучение Qwen3-0.6B для генерации русских анекдотов


In [ ]:
# используем только 1 GPU чтобы избежать проблем с DataParallel
# т к на кагле 2 карты t4 дают, меньше нельзя
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
!pip install -q transformers peft trl accelerate datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.4/517.4 kB 11.2 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 105.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 40.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 76.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 82.2 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import torch
print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0)}")

PyTorch: 2.6.0+cu124
CUDA: True
GPU: Tesla T4


## 1. Загрузка модели

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, get_peft_model

MODEL_NAME = "Qwen/Qwen3-0.6B"

# загрузка без квантизации - 0.6B влезет в T4
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    device_map="cuda:0",
    trust_remote_code=True,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

print(f"Модель загружена")
print(f"Memory: {torch.cuda.memory_allocated()/1e9:.2f} GB")

2025-12-14 18:45:59.341894: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765737959.737790      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765737959.852295      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Модель загружена
Memory: 1.19 GB


In [5]:
!huggingface-cli download Qwen/Qwen3-0.6B --local-dir /kaggle/working/qwen3_model
!cd /kaggle/working && zip -r qwen3_model.zip qwen3_model/

⚠️  Warning: 'huggingface-cli download' is deprecated. Use 'hf download' instead.
Fetching 10 files:   0%|                                 | 0/10 [00:00<?, ?it/s]Downloading 'LICENSE' to '/kaggle/working/qwen3_model/.cache/huggingface/download/DhCjcNQuMpl4FL346qr3tvNUCgY=.6634c8cc3133b3848ec74b9f275acaaa1ea618ab.incomplete'

LICENSE: 11.3kB [00:00, 26.1MB/s]
Download complete. Moving file to /kaggle/working/qwen3_model/LICENSE

README.md: 14.0kB [00:00, 36.2MB/s]
Download complete. Moving file to /kaggle/working/qwen3_model/README.md

.gitattributes: 1.57kB [00:00, 11.7MB/s]
Download complete. Moving file to /kaggle/working/qwen3_model/.gitattributes
Fetching 10 files: 100%|████████████████████████| 10/10 [00:01<00:00,  8.23it/s]
/kaggle/working/qwen3_model
  adding: qwen3_model/ (stored 0%)
  adding: qwen3_model/LICENSE (deflated 65%)
  adding: qwen3_model/.gitattributes (deflated 87%)
  adding: qwen3_model/generation_config.json (deflated 44%)
  adding: qwen3_model/.cache/ (stored 0%

In [6]:
# LoRA
lora_config = LoraConfig(
    r=64,
    lora_alpha=64,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model.enable_input_require_grads()
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 18,350,080 || all params: 614,400,000 || trainable%: 2.9867


## 2. Подготовка данных

In [7]:
from datasets import load_dataset
import re

dataset = load_dataset("igorktech/anekdots", split="train")
print(f"Всего: {len(dataset):,}")

def clean_joke(text):
    if not text or not isinstance(text, str):
        return None
    text = re.sub(r'<[^>]+>', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    if len(text) < 50 or len(text) > 500:
        return None
    if len(re.findall('[а-яА-ЯёЁ]', text)) < len(text) * 0.5:
        return None
    return text

def filter_fn(ex):
    if clean_joke(ex['text']) is None:
        return False
    ldr = ex.get('LDR')
    if ldr is not None and ldr < 0.55:
        return False
    return True

filtered = dataset.filter(filter_fn, num_proc=2)
print(f"После фильтрации: {len(filtered):,}")

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/122M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/497596 [00:00<?, ? examples/s]

Всего: 497,596


Filter (num_proc=2):   0%|          | 0/497596 [00:00<?, ? examples/s]

После фильтрации: 239,882


In [ ]:
filtered = filtered.sort('LDR', reverse=True)
train_data = filtered.select(range(min(15000, len(filtered))))
print(f"Обучающая выборка: {len(train_data):,}")

Обучающая выборка: 15,000


In [ ]:
def format_example(ex):
    joke = clean_joke(ex['text']) or ex['text'].strip()
    messages = [
        {"role": "user", "content": "Расскажи анекдот."},
        {"role": "assistant", "content": joke}
    ]
    text = tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=False
    )
    return {"text": text}

formatted_dataset = train_data.map(format_example, num_proc=2)
print(f"Данные готовы")
print(f"Пример: {formatted_dataset[0]['text'][:300]}")

Map (num_proc=2):   0%|          | 0/15000 [00:00<?, ? examples/s]

Данные готовы
Пример: <|im_start|>user
Расскажи анекдот.<|im_end|>
<|im_start|>assistant
<think>

</think>

Пришел мужик к любовнице и только легли в постель, возвращается муж. Ну любовница мужику: - прыгай в окно! А тот: -что я скамоубийца с десятого этажа прыгать. Повишу до утра утром муж уйдет. Провисел всю ночь. утро


## 3. Обучение

In [10]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

# Токенизация датасета
def tokenize_fn(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        max_length=512,
        padding="max_length",
    )

tokenized_dataset = formatted_dataset.map(
    tokenize_fn,
    batched=True,
    remove_columns=formatted_dataset.column_names,
    num_proc=2,
)

print(f"Токенизация завершена")

Map (num_proc=2):   0%|          | 0/15000 [00:00<?, ? examples/s]

Токенизация завершена


In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

training_args = TrainingArguments(
    output_dir="./output",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    
    num_train_epochs=4,          
    learning_rate=1e-4,           
    
    fp16=True,
    logging_steps=50,
    save_strategy="epoch",       
    
    optim="adamw_torch",
    warmup_steps=100,            
    lr_scheduler_type="cosine",
    weight_decay=0.01,          
    max_grad_norm=0.3,         
    
    report_to="none",
    seed=42,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
)

print("Trainer готов")

The model is already on multiple devices. Skipping the move to device specified in `args`.


Trainer готов


In [12]:
print("Начинаем обучение...")
trainer.train()
print("\nОбучение завершено!")

Начинаем обучение...


Step,Training Loss
50,3.127200
100,2.384900
150,2.301000
200,2.242600
250,2.237800
300,2.232900
350,2.231600
400,2.197800
450,2.218100
500,2.248600



Обучение завершено!


## 4. Генерация анекдотов

In [13]:
model.eval()

def generate_joke(temperature=0.8, top_p=0.9):
    messages = [{"role": "user", "content": "Расскажи анекдот."}]
    
    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=200,
            temperature=temperature,
            top_p=top_p,
            do_sample=True,
            repetition_penalty=1.15,
            pad_token_id=tokenizer.eos_token_id,
        )
    
    new_tokens = outputs[0][inputs['input_ids'].shape[1]:]
    text = tokenizer.decode(new_tokens, skip_special_tokens=True)
    text = re.sub(r'<think>.*?</think>', '', text, flags=re.DOTALL)
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

print("Тестовые анекдоты:")
print("="*60)
for i in range(10):
    print(f"\n{i+1}. {generate_joke()}")

Тестовые анекдоты:

1. Если у тебя нет денег, ты можешь работать на дураках с их роскошью и жить в магазине. Если же у тебя много денег - выигрываешь миллион за то, чтобы не поздно отдавать деньги! Вот почему я хочу стать финансовым борём. Наверное, есть ещё какие-то методы? Я такая честная лояльность, что если мне посоветовать один такой метод, сразу прибегаю к тому, который уже известен. Ну вот сейчас, какую идею вы хотите? А какая твой цель? (p.s. это можно было сделать без денег!) Дайте ответ!!!) (p.p.s.) Сделайте этот вопрос так, чтобы он был бы более нелепо...) Что делать, когда вы хотите заблокировать компьютер, но не можете найти правильную кнопку?...

2. Вопрос: Сколько пять-шесть лет в космосе, а сто лет на Земле? Ответ: 500 лет. Всё равно, не надо туда бегать! Материя всё ещё находится на своем месте... Или же все время оставаться в живых и жить на своём месте... Действительно, везде! А вот наверно можно и туда попадать... Так что если ты думаешь, что мы должны пребывать на 

## 5. Сохранение модели

In [ ]:
model.save_pretrained("/kaggle/working/qwen3_jokes_lora")
tokenizer.save_pretrained("/kaggle/working/qwen3_jokes_lora")

In [ ]:
!cd /kaggle/working && zip -r qwen3_jokes_lora.zip qwen3_jokes_lora/